# read files

In [1]:
import pymssql
import pymysql as mysql
import os
import re
import pandas as pd
import datetime
import requests

In [27]:
class stt_server:

    def __init__(self, cpu_id):

        # settings ++
        self.cpu_id = cpu_id
        self.cpu_cores = [i for i in range(0,15)]

        # ms sql
        self.sql_name = 'voice_ai'
        self.sql_server = '10.2.4.124'
        self.sql_login = 'ICECORP\\1c_sql'

        # mysql
        self.mysql_name = {
            1: 'MICO_96',
            2: 'asterisk',
        }
        self.mysql_server = '10.2.4.146'
        self.mysql_login = 'asterisk'

        self.script_path = '/home/alex/projects/call_centre_stt_server/'
        self.model_path = '/home/alex/projects/vosk-api/python/example/model'
        self.source_id = 0
        self.sources = {
            'call': 1,
            'master': 2,
        }
        self.original_storage_path = {
            1: '/mnt/share/audio/MSK_SRVCALL/RX_TX/',
            2: '/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/'
        }
        self.temp_file_path = self.script_path+'files/'
        # settings --

        self.temp_file_name = ''
        self.original_file_path = ''
        self.original_file_duration	= 0
        self.date_y = ''
        self.date_m = ''
        self.date_d = ''
        self.rec_date = ''

        #store pass in file, to prevent pass publication on gitdelete_current_queue
        with open(self.script_path+'sql.pass','r') as file:
            self.sql_pass = file.read().replace('\n', '')
            file.close()

        with open(self.script_path+'mysql.pass','r') as file:
            self.mysql_pass = file.read().replace('\n', '')
            file.close()

        self.conn = self.connect_sql()
        self.mysql_conn = {
            1: self.connect_mysql(1),
            2: self.connect_mysql(2),
        }
        
    def perf_log(self, step, spent_time, duration, linkedid):
        
        current_date = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        cursor = self.conn.cursor()
        sql_query = "insert into perf_log(event_date, step, time, cpu, file_name, duration, linkedid, source_id) "
        sql_query += "values ("
        sql_query += "'" + current_date + "', "
        sql_query += str(step) + ", "
        sql_query += str(spent_time) + ", "
        sql_query += str(self.cpu_id) + ", "
        sql_query += "'" + self.temp_file_name + "', "
        sql_query += "'" + str(duration) + "', "
        sql_query += "'" + str(linkedid) + "', "
        sql_query += "'" + str(self.source_id) + "');"
        print(sql_query)
        cursor.execute(sql_query)
        server_object.conn.commit()

    def connect_sql(self):

        return pymssql.connect(
            server = self.sql_server,
            user = self.sql_login,
            password = self.sql_pass,
            database = self.sql_name,
            #autocommit=True
        )

    def connect_mysql(self, source_id):

        return mysql.connect(
            host = self.mysql_server, 
            user = self.mysql_login, 
            passwd = self.mysql_pass,
            db = self.mysql_name[source_id],
            #autocommit = True
        )

    def get_fs_files_list(self, queue):
        
        fd_list = []

        if self.source_id == self.sources['call']:
            #print('call', len(queue), self.original_file_path)
            for root, dirs, files in os.walk(self.original_storage_path[self.source_id]):
                #find_files += [os.path.join(root, name) for name in files if name[-4:] == '.wav'] 
                for filename in files:
                    if filename[-4:] == '.wav' and not filename in queue:
                        rec_source_date = re.findall(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}', filename)
                        if len(rec_source_date) and len(rec_source_date[0]):
                            rec_date = rec_source_date[0][:10] + ' ' + rec_source_date[0][11:].replace('-', ':')

                            if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', rec_date)) == 0:
                                rec_date = 'Null'
                                print('0 Unable to extract date:', root, filename)

                            date_string  = re.findall(r'\d{4}-\d{2}-\d{2}', filename)
                            if len(date_string):
                                self.date_y = date_string[0][:4]
                                self.date_m = date_string[0][5:-3]
                                self.date_d = date_string[0][-2:]
                                #print(date_y, date_m, date_d, filename)
                                linkedid, dst, src = self.linkedid_by_filename(filename) # cycled query

                                fd_list.append({
                                    'filepath': root,
                                    'filename': filename,
                                    'rec_date': rec_date,
                                    'src': src,
                                    'dst': dst,
                                    'linkedid': linkedid,
                                })
                        else:
                            print('1 Unable to extract date:', root, filename)
        
        elif self.source_id == self.sources['master']:
            files_list = []
            for (dirpath, dirnames, filenames) in os.walk(self.original_storage_path[self.source_id]):
                files_list.extend(filenames)
                break

            # get record date
            for filename in files_list:
                if not filename in queue:
                    rec_date = 'Null'

                    #elif self.source_id == self.sources['master']:
                    uniqueid = re.findall(r'^\d*.\d*', filename)[0]
                    cursor = self.mysql_conn[self.source_id].cursor()
                    query = "select calldate, src, dst from cdr where uniqueid = '" + uniqueid + "' limit 1;"
                    cursor.execute(query)  # cycled query
                    src = ''
                    dst = ''
                    linkedid = uniqueid
                    for row in cursor.fetchall():
                        rec_date = str(row[0])
                        src = str(row[1])
                        dst = str(row[2])
                    if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', rec_date)) == 0:
                        rec_date = 'Null'
                        print('Unable to extract date from filename', filename)

                    fd_list.append({
                        'filepath': self.original_storage_path[self.source_id],
                        'filename': filename,
                        'rec_date': rec_date,
                        'src': src,
                        'dst': dst,
                        'linkedid': linkedid,
                    })

        df = pd.DataFrame(fd_list, columns = ['filepath', 'filename', 'rec_date', 'src', 'dst', 'linkedid'])
        df.sort_values(['rec_date', 'filename'], ascending=True, inplace=True)

        return df.values
    
    def get_sql_complete_files(self):

        cursor = self.conn.cursor()
        sql_query = "select distinct filename from queue where"
        sql_query += " source_id='" + str(self.source_id) + "'"
        sql_query += " order by filename;"
        cursor.execute(sql_query)
        complete_files = []
        for row in cursor.fetchall():
            complete_files.append(row[0])

        return complete_files
    
    def get_source_id(self, source_name):
        for source in self.sources.items():
            if source[0] == source_name:
                return source[1]
        return 0
    
    def set_today_ymd(self):
        self.date_y	= datetime.datetime.today().strftime('%Y')
        self.date_m	= datetime.datetime.today().strftime('%m')
        self.date_d	= datetime.datetime.today().strftime('%d')
        
    def linkedid_by_filename(self, original_file_name):

        filename = original_file_name.replace('rxtx.wav', '')

        
        #print('linkedid_by_filename', self.date_y, self.date_m, self.date_d, original_file_name)
        
        date_from = datetime.datetime(int(self.date_y), int(self.date_m), int(self.date_d))
        date_toto = date_from+datetime.timedelta(days=1)
        date_from = datetime.datetime.strptime(str(date_from), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%dT%H:%M:%S')
        date_toto = datetime.datetime.strptime(str(date_toto), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%dT%H:%M:%S')

        mysql_conn = self.connect_mysql(self.source_id)

        with mysql_conn:
        #with self.mysql_conn[self.source_id]:
            query = """
            select 
                linkedid,
                SUBSTRING(dstchannel, 5, 4),
                src
                from PT1C_cdr_MICO as PT1C_cdr_MICO
                where 
                    calldate>'"""+date_from+"""' and 
                    calldate<'"""+date_toto+"""' and 
                    PT1C_cdr_MICO.recordingfile LIKE '%"""+filename+"""%' 
                    limit 1;"""

            #cursor = mysql_conn[self.source_id].cursor()
            cursor = mysql_conn.cursor()
            cursor.execute(query)
            for row in cursor.fetchall():
                linkedid, dstchannel, src = row[0], row[1], row[2]
                #print('linkedid, dstchannel', linkedid, dstchannel)
                return linkedid, dstchannel, src
        return '', '', ''
    
    def get_fs_files_list(self, queue):

        fd_list = []

        if self.source_id == self.sources['call']:
            for root, dirs, files in os.walk(self.original_storage_path[self.source_id]):
                for filename in files:
                    if filename[-4:] == '.wav' and not filename in queue:
                        rec_source_date = re.findall(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}', filename)
                        if len(rec_source_date) and len(rec_source_date[0]):
                            rec_date = rec_source_date[0][:10] + ' ' + rec_source_date[0][11:].replace('-', ':')

                            if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', rec_date)) == 0:
                                rec_date = 'Null'
                                print('0 Unable to extract date:', root, filename)

                            date_string = re.findall(r'\d{4}-\d{2}-\d{2}', filename)
                            if len(date_string):
                                date_y = date_string[0][:4]
                                date_m = date_string[0][5:-3]
                                date_d = date_string[0][-2:]
                                linkedid, dst, src = self.linkedid_by_filename(filename, date_y, date_m, date_d)  # cycled query

                                fd_list.append({
                                    'filepath': root+'/',
                                    'filename': filename,
                                    'rec_date': rec_date,
                                    'src': src,
                                    'dst': dst,
                                    'linkedid': linkedid,
                                })
                        else:
                            print('1 Unable to extract date:', root, filename)
                # break # ToDo: remove

        elif self.source_id == self.sources['master']:
            #print('d1 master')
            files_list = []
            for (dirpath, dirnames, filenames) in os.walk(self.original_storage_path[self.source_id]):
                files_list.extend(filenames)
                
            #print('d2 os.walk')
            # get record date
            files_extracted = 0
            files_withoud_cdr_data = 0
            for filename in files_list:
                if not filename in queue:
                    rec_date = 'Null'
                    uniqueid = re.findall(r'^\d*.\d*', filename)[0]
                    cursor = self.mysql_conn[self.source_id].cursor()
                    query = "select calldate, src, dst from cdr where uniqueid = '" + uniqueid + "' limit 1;"
                    cursor.execute(query)  # cycled query
                    src = ''
                    dst = ''
                    linkedid = uniqueid
                    for row in cursor.fetchall():
                        rec_date = str(row[0])
                        src = str(row[1])
                        dst = str(row[2])
                    if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', rec_date)) == 0:
                        rec_date = 'Null'
                        #print('Unable to extract date from filename', filename)
                        files_withoud_cdr_data += 1
                        continue

                    fd_list.append({
                        'filepath': self.original_storage_path[self.source_id],
                        'filename': filename,
                        'rec_date': rec_date,
                        'src': src,
                        'dst': dst,
                        'linkedid': linkedid,
                    })
                    files_extracted += 1
                    
            print('master extracted:', files_extracted, 'without cdr data:', files_withoud_cdr_data)

        df = pd.DataFrame(fd_list, columns=['filepath', 'filename', 'rec_date', 'src', 'dst', 'linkedid'])
        df.sort_values(['rec_date', 'filename'], ascending=True, inplace=True)
        
        

        return df.values

### master date by linkedid fix ++

In [20]:
server_object = stt_server(0)
server_object.source_id = 2

In [5]:
cursor = server_object.conn.cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

('transcribation_date',)
('date_y',)
('date_m',)
('date_d',)
('side',)
('text',)
('start',)
('audio_file_name',)
('conf',)
('end_time',)
('sentiment',)
('sentiment_pos',)
('sentiment_neg',)
('ID',)
('linkedid',)
('dst',)
('record_date',)
('source_id',)
('src',)
('cpu_id',)
('duration',)


In [7]:
# last transcribed file whithout record_date
cursor = server_object.conn.cursor()

query = 'select top 1 transcribation_date, record_date, linkedid, audio_file_name, duration'
query += ' from transcribations where record_date is Null'
query += ' order by transcribation_date desc;'

cursor.execute(query)
for row in cursor.fetchall():
    print(row)

(datetime.datetime(2021, 3, 10, 7, 21, 58), None, '1615349750.2104256', '1615349750.2104256-in.wav', 7.36)


In [8]:
# cdr info by linkedid
uniqueid = '1615349750.2104256'
cursor = server_object.mysql_conn[server_object.source_id].cursor()
query = "select calldate from cdr where uniqueid = '"+uniqueid+"' limit 1;"
cursor.execute(query)
for row in cursor.fetchall():
    #rec_date = str(row[0])
    print(row[0])

2021-03-10 07:15:50


In [15]:
server_object.source_id == server_object.sources['master']

True

In [28]:
server_object = stt_server(0)
server_object.source_id = 2
complete_files = []
for filepath, filename, rec_date, src, dst, linkedid in server_object.get_fs_files_list(complete_files):
    print(filename, rec_date, linkedid)

master extracted: 4659 without cdr data: 10
1615288348.2092415-in.wav 2021-03-09 14:12:28 1615288348.2092415
1615288363.2092432-out.wav 2021-03-09 14:12:43 1615288363.2092432
1615288430.2092487-in.wav 2021-03-09 14:13:50 1615288430.2092487
1615288463.2092518-out.wav 2021-03-09 14:14:23 1615288463.2092518
1615288482.2092547-in.wav 2021-03-09 14:14:42 1615288482.2092547
1615288494.2092565-out.wav 2021-03-09 14:14:54 1615288494.2092565
1615288542.2092607-in.wav 2021-03-09 14:15:42 1615288542.2092607
1615288569.2092636-out.wav 2021-03-09 14:16:09 1615288569.2092636
1615288625.2092680-out.wav 2021-03-09 14:17:05 1615288625.2092680
1615288734.2092762-out.wav 2021-03-09 14:18:54 1615288734.2092762
1615288741.2092775-out.wav 2021-03-09 14:19:01 1615288741.2092775
1615288767.2092793-out.wav 2021-03-09 14:19:27 1615288767.2092793
1615288770.2092799-in.wav 2021-03-09 14:19:30 1615288770.2092799
1615288772.2092801-in.wav 2021-03-09 14:19:32 1615288772.2092801
1615288775.2092805-in.wav 2021-03-09 1

1615355909.2105076-out.wav 2021-03-10 08:58:29 1615355909.2105076
1615355958.2105080-in.wav 2021-03-10 08:59:18 1615355958.2105080
1615355958.2105080-out.wav 2021-03-10 08:59:18 1615355958.2105080
1615355977.2105084-in.wav 2021-03-10 08:59:37 1615355977.2105084
1615355977.2105084-out.wav 2021-03-10 08:59:37 1615355977.2105084
1615355997.2105086-in.wav 2021-03-10 08:59:57 1615355997.2105086
1615355997.2105086-out.wav 2021-03-10 08:59:57 1615355997.2105086
1615355999.2105090-in.wav 2021-03-10 08:59:59 1615355999.2105090
1615355999.2105090-out.wav 2021-03-10 08:59:59 1615355999.2105090
1615356012.2105094-in.wav 2021-03-10 09:00:12 1615356012.2105094
1615356012.2105094-out.wav 2021-03-10 09:00:12 1615356012.2105094
1615356017.2105098-in.wav 2021-03-10 09:00:17 1615356017.2105098
1615356017.2105098-out.wav 2021-03-10 09:00:17 1615356017.2105098
1615356053.2105102-in.wav 2021-03-10 09:00:53 1615356053.2105102
1615356053.2105102-out.wav 2021-03-10 09:00:53 1615356053.2105102
1615356061.210510

1615361623.2108203-in.wav 2021-03-10 10:33:43 1615361623.2108203
1615361623.2108203-out.wav 2021-03-10 10:33:43 1615361623.2108203
1615361630.2108212-in.wav 2021-03-10 10:33:50 1615361630.2108212
1615361630.2108212-out.wav 2021-03-10 10:33:50 1615361630.2108212
1615361647.2108226-in.wav 2021-03-10 10:34:07 1615361647.2108226
1615361647.2108226-out.wav 2021-03-10 10:34:07 1615361647.2108226
1615361663.2108241-in.wav 2021-03-10 10:34:23 1615361663.2108241
1615361663.2108241-out.wav 2021-03-10 10:34:23 1615361663.2108241
1615361666.2108249-in.wav 2021-03-10 10:34:26 1615361666.2108249
1615361666.2108249-out.wav 2021-03-10 10:34:26 1615361666.2108249
1615361667.2108251-in.wav 2021-03-10 10:34:27 1615361667.2108251
1615361667.2108251-out.wav 2021-03-10 10:34:27 1615361667.2108251
1615361669.2108253-in.wav 2021-03-10 10:34:29 1615361669.2108253
1615361669.2108253-out.wav 2021-03-10 10:34:29 1615361669.2108253
1615361669.2108255-in.wav 2021-03-10 10:34:29 1615361669.2108255
1615361669.2108255

1615374830.2115874-out.wav 2021-03-10 14:13:50 1615374830.2115874
1615374847.2115884-in.wav 2021-03-10 14:14:07 1615374847.2115884
1615374847.2115884-out.wav 2021-03-10 14:14:07 1615374847.2115884
1615374850.2115886-in.wav 2021-03-10 14:14:10 1615374850.2115886
1615374850.2115886-out.wav 2021-03-10 14:14:10 1615374850.2115886
1615374860.2115896-in.wav 2021-03-10 14:14:20 1615374860.2115896
1615374860.2115896-out.wav 2021-03-10 14:14:20 1615374860.2115896
1615374876.2115906-in.wav 2021-03-10 14:14:36 1615374876.2115906
1615374876.2115906-out.wav 2021-03-10 14:14:36 1615374876.2115906
1615374878.2115910-in.wav 2021-03-10 14:14:38 1615374878.2115910
1615374878.2115910-out.wav 2021-03-10 14:14:38 1615374878.2115910
1615374889.2115916-in.wav 2021-03-10 14:14:49 1615374889.2115916
1615374889.2115916-out.wav 2021-03-10 14:14:49 1615374889.2115916
1615374897.2115928-in.wav 2021-03-10 14:14:57 1615374897.2115928
1615374897.2115928-out.wav 2021-03-10 14:14:57 1615374897.2115928
1615374905.211593

1615380087.2119316-out.wav 2021-03-10 15:41:27 1615380087.2119316
1615380098.2119322-in.wav 2021-03-10 15:41:38 1615380098.2119322
1615380098.2119322-out.wav 2021-03-10 15:41:38 1615380098.2119322
1615380101.2119324-in.wav 2021-03-10 15:41:41 1615380101.2119324
1615380101.2119324-out.wav 2021-03-10 15:41:41 1615380101.2119324
1615380105.2119328-in.wav 2021-03-10 15:41:45 1615380105.2119328
1615380105.2119328-out.wav 2021-03-10 15:41:45 1615380105.2119328
1615380113.2119338-in.wav 2021-03-10 15:41:53 1615380113.2119338
1615380113.2119338-out.wav 2021-03-10 15:41:53 1615380113.2119338
1615380115.2119340-in.wav 2021-03-10 15:41:55 1615380115.2119340
1615380115.2119340-out.wav 2021-03-10 15:41:55 1615380115.2119340
1615380118.2119346-in.wav 2021-03-10 15:41:58 1615380118.2119346
1615380118.2119346-out.wav 2021-03-10 15:41:58 1615380118.2119346
1615380122.2119348-in.wav 2021-03-10 15:42:02 1615380122.2119348
1615380122.2119348-out.wav 2021-03-10 15:42:02 1615380122.2119348
1615380128.211935

### master date by linkedid fix --

In [27]:
import time

In [39]:
server_object = stt_server(0)
cursor = server_object.conn.cursor()
sql_query = "select filepath, filename, duration, source_id, "
sql_query += "record_date, src, dst, linkedid from queue "
sql_query += "where cpu_id='"+str(server_object.cpu_id)+"' "
sql_query += "and source_id = '1' " # ToDo: remove
sql_query += "order by ISNULL(record_date, 0) desc, record_date, linkedid, filename;"
processed = 0
cursor.execute(sql_query)
linkedid = ''
for row in cursor.fetchall():

    #queue_start = datetime.datetime.now()
    queue_start = time.time()

    original_file_path = row[0]
    original_file_name = row[1]
    original_file_duration = row[2]
    server_object.source_id = row[3]
    rec_date = row[4]
    src = row[5]
    dst = row[6]
    linkedid = row[7]

    files_converted = 0

    if not os.path.isfile(original_file_path + original_file_name):
        msg = 'File not found: '+ original_file_path + original_file_name
        msg += '\nRemoving from queue..'
        print(msg)
        #server_object.send_to_telegram(msg)
        #server_object.delete_current_queue(original_file_name, linkedid)
        continue
    print(original_file_path + original_file_name, 'exists')
    break
print('k')

File not found: /mnt/share/audio/MSK_SRVCALL/RX_TX/RXTX_2021-03/09/in_8125609584_2021-03-09-13-54-03rxtx.wav
Removing from queue..
/mnt/share/audio/MSK_SRVCALL/RX_TX/RXTX_2021-03/09/in_9030102285_2021-03-09-13-51-01rxtx.wav exists
k


In [43]:
#os.path.isfile('/mnt/share/audio/MSK_SRVCALL/RX_TX/RXTX_2021-03/09/in_8125609584_2021-03-09-13-54-03rxtx.wav')
os.path.isfile('/mnt/share/audio/MSK_SRVCALL/RX_TX/RXTX_2021-03/09/in_9030102285_2021-03-09-13-51-01rxtx.wav')

True

### file remove error alert

In [8]:
chat = '106129214'

In [5]:
with open('telegram_bot.token','r') as file:
    telegram_bot_token = file.read().replace('\n', '')
    file.close()

In [15]:
def send_to_telegram(chat,message):
    headers = {
    "Origin": "https://api.telegram.org",
    "Referer": 'https://api.telegram.org/bot' + telegram_bot_token,
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'}
    #url	= "http://scriptlab.net/telegram/bots/relaybot/relaylocked.php?chat="+chat+"&text="+message
    url = 'https://api.telegram.org/bot' + telegram_bot_token
    url += '/sendMessage?chat_id=' + str(chat)
    url += '&text=' + str(message)
    requests.get(url,headers = headers)

In [16]:
#myfile = "/var/www/html/test_4.txt"
myfile = "/home/alex/test_4.txt"
try:
    os.remove(myfile)
    print('succesfully removed', myfile)
except OSError as e:  ## if failed, report it back to the user ##
    print("Error: %s - %s." % (e.filename, e.strerror))
    send_to_telegram(chat,'Unable to remove file:\n' + myfile + '\n' + str(e))

Error: /home/alex/test_4.txt - No such file or directory.


### ***

In [49]:
# select
cursor = server_object.conn.cursor()
#sql_query = "select * from perf_log;"
sql_query = "select "
sql_query += "record_date, ISNULL(record_date, 0) from queue "
#sql_query += "where record_date is Null "
sql_query += "order by ISNULL(record_date, 0) desc, record_date;"
cursor.execute(sql_query)
rows = []
for row in cursor.fetchall():
    print(row)
    rows.append(row)

(datetime.datetime(2021, 3, 4, 9, 18, 25), datetime.datetime(2021, 3, 4, 9, 18, 25))
(datetime.datetime(2021, 3, 4, 9, 18, 2), datetime.datetime(2021, 3, 4, 9, 18, 2))
(datetime.datetime(2021, 3, 4, 9, 17, 14), datetime.datetime(2021, 3, 4, 9, 17, 14))
(datetime.datetime(2021, 3, 4, 9, 16, 29), datetime.datetime(2021, 3, 4, 9, 16, 29))
(datetime.datetime(2021, 3, 4, 9, 16, 5), datetime.datetime(2021, 3, 4, 9, 16, 5))
(datetime.datetime(2021, 3, 4, 9, 16, 3), datetime.datetime(2021, 3, 4, 9, 16, 3))
(datetime.datetime(2021, 3, 4, 9, 15, 58), datetime.datetime(2021, 3, 4, 9, 15, 58))
(datetime.datetime(2021, 3, 4, 9, 15, 48), datetime.datetime(2021, 3, 4, 9, 15, 48))
(datetime.datetime(2021, 3, 4, 9, 15, 37), datetime.datetime(2021, 3, 4, 9, 15, 37))
(datetime.datetime(2021, 3, 4, 9, 15, 28), datetime.datetime(2021, 3, 4, 9, 15, 28))
(datetime.datetime(2021, 3, 4, 9, 15, 13), datetime.datetime(2021, 3, 4, 9, 15, 13))
(datetime.datetime(2021, 3, 4, 9, 14, 31), datetime.datetime(2021, 3, 4

(datetime.datetime(2021, 3, 3, 13, 49, 52), datetime.datetime(2021, 3, 3, 13, 49, 52))
(datetime.datetime(2021, 3, 3, 13, 49, 50), datetime.datetime(2021, 3, 3, 13, 49, 50))
(datetime.datetime(2021, 3, 3, 13, 49, 50), datetime.datetime(2021, 3, 3, 13, 49, 50))
(datetime.datetime(2021, 3, 3, 13, 49, 42), datetime.datetime(2021, 3, 3, 13, 49, 42))
(datetime.datetime(2021, 3, 3, 13, 49, 40), datetime.datetime(2021, 3, 3, 13, 49, 40))
(datetime.datetime(2021, 3, 3, 13, 49, 39), datetime.datetime(2021, 3, 3, 13, 49, 39))
(datetime.datetime(2021, 3, 3, 13, 49, 39), datetime.datetime(2021, 3, 3, 13, 49, 39))
(datetime.datetime(2021, 3, 3, 13, 49, 34), datetime.datetime(2021, 3, 3, 13, 49, 34))
(datetime.datetime(2021, 3, 3, 13, 49, 34), datetime.datetime(2021, 3, 3, 13, 49, 34))
(datetime.datetime(2021, 3, 3, 13, 49, 31), datetime.datetime(2021, 3, 3, 13, 49, 31))
(datetime.datetime(2021, 3, 3, 13, 49, 31), datetime.datetime(2021, 3, 3, 13, 49, 31))
(datetime.datetime(2021, 3, 3, 13, 49, 30),

(datetime.datetime(2021, 3, 2, 17, 17, 42), datetime.datetime(2021, 3, 2, 17, 17, 42))
(datetime.datetime(2021, 3, 2, 17, 17, 40), datetime.datetime(2021, 3, 2, 17, 17, 40))
(datetime.datetime(2021, 3, 2, 17, 16, 59), datetime.datetime(2021, 3, 2, 17, 16, 59))
(datetime.datetime(2021, 3, 2, 17, 16, 58), datetime.datetime(2021, 3, 2, 17, 16, 58))
(datetime.datetime(2021, 3, 2, 17, 16, 25), datetime.datetime(2021, 3, 2, 17, 16, 25))
(datetime.datetime(2021, 3, 2, 17, 15, 33), datetime.datetime(2021, 3, 2, 17, 15, 33))
(datetime.datetime(2021, 3, 2, 17, 15, 29), datetime.datetime(2021, 3, 2, 17, 15, 29))
(datetime.datetime(2021, 3, 2, 17, 15, 24), datetime.datetime(2021, 3, 2, 17, 15, 24))
(datetime.datetime(2021, 3, 2, 17, 14, 54), datetime.datetime(2021, 3, 2, 17, 14, 54))
(datetime.datetime(2021, 3, 2, 17, 14, 49), datetime.datetime(2021, 3, 2, 17, 14, 49))
(datetime.datetime(2021, 3, 2, 17, 14), datetime.datetime(2021, 3, 2, 17, 14))
(datetime.datetime(2021, 3, 2, 17, 13, 57), datetim

(datetime.datetime(2021, 3, 1, 14, 12, 23), datetime.datetime(2021, 3, 1, 14, 12, 23))
(datetime.datetime(2021, 3, 1, 14, 12, 22), datetime.datetime(2021, 3, 1, 14, 12, 22))
(datetime.datetime(2021, 3, 1, 14, 12, 22), datetime.datetime(2021, 3, 1, 14, 12, 22))
(datetime.datetime(2021, 3, 1, 14, 12, 17), datetime.datetime(2021, 3, 1, 14, 12, 17))
(datetime.datetime(2021, 3, 1, 14, 12, 14), datetime.datetime(2021, 3, 1, 14, 12, 14))
(datetime.datetime(2021, 3, 1, 14, 12, 14), datetime.datetime(2021, 3, 1, 14, 12, 14))
(datetime.datetime(2021, 3, 1, 14, 12, 14), datetime.datetime(2021, 3, 1, 14, 12, 14))
(datetime.datetime(2021, 3, 1, 14, 12, 14), datetime.datetime(2021, 3, 1, 14, 12, 14))
(datetime.datetime(2021, 3, 1, 14, 12, 6), datetime.datetime(2021, 3, 1, 14, 12, 6))
(datetime.datetime(2021, 3, 1, 14, 12, 4), datetime.datetime(2021, 3, 1, 14, 12, 4))
(datetime.datetime(2021, 3, 1, 14, 12, 4), datetime.datetime(2021, 3, 1, 14, 12, 4))
(datetime.datetime(2021, 3, 1, 14, 11, 59), datet

(datetime.datetime(2021, 3, 1, 9, 13, 49), datetime.datetime(2021, 3, 1, 9, 13, 49))
(datetime.datetime(2021, 3, 1, 9, 13, 33), datetime.datetime(2021, 3, 1, 9, 13, 33))
(datetime.datetime(2021, 3, 1, 9, 13, 1), datetime.datetime(2021, 3, 1, 9, 13, 1))
(datetime.datetime(2021, 3, 1, 9, 12, 10), datetime.datetime(2021, 3, 1, 9, 12, 10))
(datetime.datetime(2021, 3, 1, 9, 11, 59), datetime.datetime(2021, 3, 1, 9, 11, 59))
(datetime.datetime(2021, 3, 1, 9, 11, 10), datetime.datetime(2021, 3, 1, 9, 11, 10))
(datetime.datetime(2021, 3, 1, 9, 10, 59), datetime.datetime(2021, 3, 1, 9, 10, 59))
(datetime.datetime(2021, 3, 1, 9, 10, 42), datetime.datetime(2021, 3, 1, 9, 10, 42))
(datetime.datetime(2021, 3, 1, 9, 10, 34), datetime.datetime(2021, 3, 1, 9, 10, 34))
(datetime.datetime(2021, 3, 1, 9, 10), datetime.datetime(2021, 3, 1, 9, 10))
(datetime.datetime(2021, 3, 1, 9, 9, 29), datetime.datetime(2021, 3, 1, 9, 9, 29))
(datetime.datetime(2021, 3, 1, 9, 9, 14), datetime.datetime(2021, 3, 1, 9, 9,

...

(datetime.datetime(2021, 2, 24, 20, 21, 52), datetime.datetime(2021, 2, 24, 20, 21, 52))
(datetime.datetime(2021, 2, 24, 20, 20, 43), datetime.datetime(2021, 2, 24, 20, 20, 43))
(datetime.datetime(2021, 2, 24, 20, 20, 33), datetime.datetime(2021, 2, 24, 20, 20, 33))
(datetime.datetime(2021, 2, 24, 20, 20, 25), datetime.datetime(2021, 2, 24, 20, 20, 25))
(datetime.datetime(2021, 2, 24, 20, 20, 10), datetime.datetime(2021, 2, 24, 20, 20, 10))
(datetime.datetime(2021, 2, 24, 20, 20), datetime.datetime(2021, 2, 24, 20, 20))
(datetime.datetime(2021, 2, 24, 20, 19, 53), datetime.datetime(2021, 2, 24, 20, 19, 53))
(datetime.datetime(2021, 2, 24, 20, 19, 44), datetime.datetime(2021, 2, 24, 20, 19, 44))
(datetime.datetime(2021, 2, 24, 20, 19, 18), datetime.datetime(2021, 2, 24, 20, 19, 18))
(datetime.datetime(2021, 2, 24, 20, 19, 13), datetime.datetime(2021, 2, 24, 20, 19, 13))
(datetime.datetime(2021, 2, 24, 20, 18, 54), datetime.datetime(2021, 2, 24, 20, 18, 54))
(datetime.datetime(2021, 2, 2

(datetime.datetime(2021, 2, 23, 11, 4, 55), datetime.datetime(2021, 2, 23, 11, 4, 55))
(datetime.datetime(2021, 2, 23, 11, 4, 36), datetime.datetime(2021, 2, 23, 11, 4, 36))
(datetime.datetime(2021, 2, 23, 11, 3, 43), datetime.datetime(2021, 2, 23, 11, 3, 43))
(datetime.datetime(2021, 2, 23, 11, 3, 34), datetime.datetime(2021, 2, 23, 11, 3, 34))
(datetime.datetime(2021, 2, 23, 11, 3, 1), datetime.datetime(2021, 2, 23, 11, 3, 1))
(datetime.datetime(2021, 2, 23, 11, 2, 24), datetime.datetime(2021, 2, 23, 11, 2, 24))
(datetime.datetime(2021, 2, 23, 11, 1, 50), datetime.datetime(2021, 2, 23, 11, 1, 50))
(datetime.datetime(2021, 2, 23, 11, 1, 3), datetime.datetime(2021, 2, 23, 11, 1, 3))
(datetime.datetime(2021, 2, 23, 11, 0, 54), datetime.datetime(2021, 2, 23, 11, 0, 54))
(datetime.datetime(2021, 2, 23, 10, 59, 48), datetime.datetime(2021, 2, 23, 10, 59, 48))
(datetime.datetime(2021, 2, 23, 10, 59, 33), datetime.datetime(2021, 2, 23, 10, 59, 33))
(datetime.datetime(2021, 2, 23, 10, 59, 3),

(datetime.datetime(2021, 2, 21, 14, 1, 8), datetime.datetime(2021, 2, 21, 14, 1, 8))
(datetime.datetime(2021, 2, 21, 14, 0, 55), datetime.datetime(2021, 2, 21, 14, 0, 55))
(datetime.datetime(2021, 2, 21, 14, 0, 47), datetime.datetime(2021, 2, 21, 14, 0, 47))
(datetime.datetime(2021, 2, 21, 14, 0, 24), datetime.datetime(2021, 2, 21, 14, 0, 24))
(datetime.datetime(2021, 2, 21, 14, 0, 13), datetime.datetime(2021, 2, 21, 14, 0, 13))
(datetime.datetime(2021, 2, 21, 13, 59, 38), datetime.datetime(2021, 2, 21, 13, 59, 38))
(datetime.datetime(2021, 2, 21, 13, 59, 36), datetime.datetime(2021, 2, 21, 13, 59, 36))
(datetime.datetime(2021, 2, 21, 13, 59, 9), datetime.datetime(2021, 2, 21, 13, 59, 9))
(datetime.datetime(2021, 2, 21, 13, 58, 49), datetime.datetime(2021, 2, 21, 13, 58, 49))
(datetime.datetime(2021, 2, 21, 13, 58, 34), datetime.datetime(2021, 2, 21, 13, 58, 34))
(datetime.datetime(2021, 2, 21, 13, 58, 27), datetime.datetime(2021, 2, 21, 13, 58, 27))
(datetime.datetime(2021, 2, 21, 13,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




(datetime.datetime(2021, 2, 16, 13, 53, 19), datetime.datetime(2021, 2, 16, 13, 53, 19))
(datetime.datetime(2021, 2, 16, 13, 53, 17), datetime.datetime(2021, 2, 16, 13, 53, 17))
(datetime.datetime(2021, 2, 16, 13, 53, 17), datetime.datetime(2021, 2, 16, 13, 53, 17))
(datetime.datetime(2021, 2, 16, 13, 52, 58), datetime.datetime(2021, 2, 16, 13, 52, 58))
(datetime.datetime(2021, 2, 16, 13, 52, 55), datetime.datetime(2021, 2, 16, 13, 52, 55))
(datetime.datetime(2021, 2, 16, 13, 52, 55), datetime.datetime(2021, 2, 16, 13, 52, 55))
(datetime.datetime(2021, 2, 16, 13, 52, 55), datetime.datetime(2021, 2, 16, 13, 52, 55))
(datetime.datetime(2021, 2, 16, 13, 52, 50), datetime.datetime(2021, 2, 16, 13, 52, 50))
(datetime.datetime(2021, 2, 16, 13, 52, 50), datetime.datetime(2021, 2, 16, 13, 52, 50))
(datetime.datetime(2021, 2, 16, 13, 52, 27), datetime.datetime(2021, 2, 16, 13, 52, 27))
(datetime.datetime(2021, 2, 16, 13, 52, 27), datetime.datetime(2021, 2, 16, 13, 52, 27))
(datetime.datetime(2

In [31]:
len(rows)

230

In [60]:
server_object = stt_server(0)
server_object.source_id = 2
cursor = server_object.conn.cursor()

query = 'select '
query += 'cpu_id, '
query += 'sum(duration), '
query += 'count(id) as cnt '
"""query += 'max(record_date) as trans_date, '
query += 'max(transcribation_date) as rec_date '"""
query += 'from transcribations '
query += 'where not cpu_id is null '
query += 'group by cpu_id order by cpu_id;'

"""query = 'select '
query += 'cpu_id, '
query += 'count(distinct filename) as filename, '
query += 'max(record_date) as trans_date '
#query += 'max(transcribation_date) as rec_date '
query += 'from queue group by cpu_id order by cpu_id;'
"""
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

(0, 24969.55999999996, 128)
(1, 4926.919999999998, 52)
(2, 6574.420000000005, 45)
(3, 4786.860000000001, 49)
(4, 8733.160000000003, 69)
(5, 14827.299999999992, 77)
(6, 10175.54, 74)
(7, 22766.56, 38)
(8, 16021.760000000007, 64)
(9, 2963.379999999997, 57)
(10, 18047.879999999994, 54)
(11, 4111.4000000000015, 60)
(12, 2893.86, 49)
(13, 6097.759999999994, 56)
(14, 8504.520000000002, 56)
(15, 13787.759999999987, 58)
(16, 34499.080000000045, 77)
(17, 2908.8999999999987, 53)
(18, 15529.199999999988, 60)
(19, 13816.439999999988, 57)
(20, 12196.959999999988, 61)
(21, 3991.8199999999997, 41)
(22, 7416.539999999995, 63)
(23, 8700.320000000005, 62)
(24, 11277.419999999995, 57)
(25, 13973.69999999998, 65)
(26, 6924.080000000006, 54)
(27, 12595.279999999982, 61)
(28, 10292.320000000003, 61)


In [62]:
server_object = stt_server(0)
server_object.source_id = 2
cursor = server_object.conn.cursor()

query = 'select '
query += 'cpu_id, '
query += 'sum(duration), '
query += 'count(id) as cnt '
query += 'from transcribations '
query += 'where not cpu_id is null '
query += 'group by cpu_id order by cpu_id;'

cursor.execute(query)
for row in cursor.fetchall():
    print(row)

(0, 33293.21999999993, 179)
(1, 12063.480000000018, 107)
(2, 10781.14000000001, 95)
(3, 8678.560000000001, 98)
(4, 12063.219999999992, 107)
(5, 19926.199999999983, 128)
(6, 21307.52000000001, 124)
(7, 46731.360000000044, 78)
(8, 21951.619999999984, 117)
(9, 7639.68000000001, 109)
(10, 27272.160000000025, 94)
(11, 8188.899999999996, 122)
(12, 9417.379999999994, 101)
(13, 11930.859999999982, 107)
(14, 12510.699999999999, 104)
(15, 19569.019999999997, 115)
(16, 54212.840000000084, 121)
(17, 9237.539999999985, 116)
(18, 23623.679999999982, 107)
(19, 21304.439999999984, 94)
(20, 20738.579999999994, 112)
(21, 9357.040000000005, 93)
(22, 12944.339999999971, 121)
(23, 12302.600000000026, 107)
(24, 21568.519999999997, 118)
(25, 22194.439999999962, 112)
(26, 13093.999999999998, 105)
(27, 16315.359999999973, 102)
(28, 14569.300000000008, 108)


In [9]:
server_object = stt_server(0)
server_object.source_id = 2
cursor = server_object.conn.cursor()
query = 'select top 10'
#query += 'cpu_id, '
#query += 'count(distinct audio_file_name) as filename, '
query += 'side, source_id, record_date as trans_date, '
query += 'transcribation_date as rec_date '
query += 'from transcribations order by transcribation_date desc;'
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

(True, 2, None, datetime.datetime(2021, 3, 4, 10, 1, 46))
(True, 2, None, datetime.datetime(2021, 3, 4, 10, 1, 10))
(True, 2, None, datetime.datetime(2021, 3, 4, 10, 0, 58))
(True, 2, None, datetime.datetime(2021, 3, 4, 10, 0, 58))
(True, 2, None, datetime.datetime(2021, 3, 4, 10, 0, 58))
(True, 2, None, datetime.datetime(2021, 3, 4, 10, 0, 58))
(True, 2, None, datetime.datetime(2021, 3, 4, 10, 0, 58))
(True, 2, None, datetime.datetime(2021, 3, 4, 10, 0, 58))
(True, 2, None, datetime.datetime(2021, 3, 4, 10, 0, 58))
(True, 2, None, datetime.datetime(2021, 3, 4, 10, 0, 57))


### perf_log

In [64]:
server_object = stt_server(0)
server_object.source_id = 2

cursor = server_object.conn.cursor()

sql_query = "select top 4 transcribation_date, count(distinct audio_file_name) from transcribations group by transcribation_date, audio_file_name order by transcribation_date desc"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

(datetime.datetime(2021, 3, 4, 13, 12, 27), 1)
(datetime.datetime(2021, 3, 4, 13, 12, 22), 1)
(datetime.datetime(2021, 3, 4, 13, 12, 21), 1)
(datetime.datetime(2021, 3, 4, 13, 12, 21), 1)


### performance monitoring

In [3]:
server_object = stt_server(0)
server_object.source_id = 2

In [16]:
# select
cursor = server_object.conn.cursor()
#sql_query = "select * from perf_log;"
sql_query = "select top 40 filepath, filename, duration, source_id, "
sql_query += "record_date, src, dst, linkedid from queue "
sql_query += "where cpu_id='"+str(server_object.cpu_id)+"' "
sql_query += "order by record_date, filename;"
cursor.execute(sql_query)
rows = []
for row in cursor.fetchall():
    print(row)
    rows.append(row)

('/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/', '1614753605.1975710-in.wav', 72.3, 2, None, '', '', '1614753605.1975710')
('/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/', '1614755503.1976672-out.wav', 170.1, 2, None, '', '', '1614755503.1976672')
('/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/', '1614759911.1979512-out.wav', 238.1, 2, None, '', '', '1614759911.1979512')
('/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/', '1614760260.1979748-out.wav', 40.42, 2, None, '', '', '1614760260.1979748')
('/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/', '1614760480.1979883-out.wav', 28.92, 2, None, '', '', '1614760480.1979883')
('/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/', '1614760744.1980019-in.wav', 15.04, 2, None, '', '', '1614760744.1980019')
('/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/', '1614762881.1981180-in.wav', 33.88, 2, None, '', '', '1614762881.1981180')
('/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/', '1614763797.1981716-in.wav', 126.32, 2, None, '', '', '1614763797.1981716')
('/mnt/share/audio/MSK_SRVCALL/REC_I

In [10]:
len(rows)

1926

In [4]:
server_object = stt_server(0)
server_object.source_id = 2
cursor = server_object.conn.cursor()

In [56]:
# Delete from perf_log
server_object = stt_server(0)
sql_query = "delete from perf_log;"
cursor = server_object.conn.cursor()
#cursor.execute(sql_query)
#server_object.conn.commit() # autocommit

In [75]:
query = "select event_date from perf_log where cores = 9 and event_date = '2021-03-05 14:49:12';"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

In [76]:
server_object = stt_server(0)
sql_query = "delete from perf_log where cores = 9 and event_date = '2021-03-05 14:49:12';"
cursor = server_object.conn.cursor()
#cursor.execute(sql_query)
#server_object.conn.commit() # autocommit

In [58]:
# select from perflog
sql_query = "select top 10 time, duration, linkedid from perf_log where step = 2;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

(12.394242525100708, 41.14, '1614936540.2023716')


In [10]:
# perf_log stat: transcribing
sql_query = "select avg(time), avg(duration), count(linkedid) from perf_log "
sql_query += "where step = 1 and source_id = 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

(440.6440496997757, 128.1874653626724, 9816)


In [45]:
# perf_log stat: transcribing
sql_query = "select avg(time), avg(duration), count(linkedid) from perf_log "
sql_query += "where step = 2 and source_id = 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

(None, None, 0)


In [129]:
# insert func
server_object.perf_log(2, 4, 30, '9879.378429')

insert into perf_log(event_date, step, time, cpu, file_name, duration, linkedid, source_id) values ('2021-03-04 15:28:57', 2, 4, 0, '', '30', '9879.378429', '2');


In [29]:
import time

In [32]:
trans_start = datetime.datetime.now()
time.sleep(2.5)
trans_end = datetime.datetime.now()
trans_diff_ms = (trans_end - trans_start)#.microseconds / 1000000
'ms', trans_diff_ms.seconds# + trans_diff_ms.microseconds / 1000000b

('ms', 2)

In [38]:
import time
start = time.time()

time.sleep(1.3)  # or do something more productive

done = time.time()
elapsed = done - start
print(elapsed)

1.3016088008880615


### last transcribed

In [ ]:
server_object = stt_server(0)
server_object.source_id = 2

cursor = server_object.conn.cursor()

#sql_query = "select audio_file_name, linkedid, record_date, start, side, text from transcribations where source_id = '2' order by record_date, start, side;"
sql_query = "select top 4 linkedid, transcribation_date, record_date, audio_file_name from transcribations "
sql_query += "where linkedid = '1614577576.20753637'"
#sql_query += "where source_id = '2' and "
#sql_query += "transcribation_date<'2021-02-15 12:00:00' "
sql_query += "order by record_date desc, start desc;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

### date repair

### check date cdr

In [ ]:
uniqueid = '1612432458.1443773'
cursor = server_object.mysql_conn[server_object.source_id].cursor()
query = "select calldate from cdr where uniqueid = '"+uniqueid+"' limit 1;"
cursor.execute(query)
for row in cursor.fetchall():
    #rec_date = str(row[0])
    print(row[0])

### file list 1

In [ ]:
server_object = stt_server(0)
server_object.source_id = 1

In [ ]:
complete_files = server_object.get_sql_complete_files()
len(complete_files)

In [ ]:
new_files = server_object.get_fs_files_list(complete_files)
len(new_files)

In [ ]:
new_files[0]

### file list 2

In [ ]:
server_object = stt_server(0)
server_object.source_id = 2

In [ ]:
complete_files_2 = server_object.get_sql_complete_files()
len(complete_files_2)

In [ ]:
new_files_2 = server_object.get_fs_files_list(complete_files_2)
len(new_files_2)

In [ ]:
new_files_2[0]

### next

In [ ]:
def find_files(catalog):
    fd_list = []
    for root, dirs, files in os.walk(catalog):        
        #find_files += [os.path.join(root, name) for name in files if name[-4:] == '.wav'] 
        for filename in files:
            if filename[-4:] == '.wav':
                #print(filename)
                rec_source_date = re.findall(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}', filename)
                if len(rec_source_date) and len(rec_source_date[0]):
                    rec_date = rec_source_date[0][:10] + ' ' + rec_source_date[0][11:].replace('-', ':')
                    
                    if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', rec_date)) == 0:
                        rec_date = 'Null'
                        print('0 Unable to extract date:', root, filename)

                    fd_list.append({
                        'filepath': root,
                        'filename': filename,
                        'rec_date': rec_date
                    })
                else:
                    print('1 Unable to extract date:', root, filename)
    return fd_list

In [ ]:
fs_list = find_files(server_object.original_storage_path[server_object.source_id])
len(fs_list), fs_list[0]

### remove complete files

In [ ]:
transcribed = []
server_object = stt_server(0)
server_object.source_id = 1
cursor = server_object.conn.cursor()
sql_query = "select distinct audio_file_name as filename, date_y, date_m, date_d"
sql_query += " from transcribations where source_id=1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    transcribed.append({
                'filename': row[0],
                'date_y': row[1],
                'date_m': row[2],
                'date_d': row[3],
            })
len(transcribed), transcribed[0]

In [ ]:
full_path = server_object.original_storage_path[1]
full_path += 'RXTX_' + transcribed[0]['date_y']
full_path += '-' + transcribed[0]['date_m'] + '/'
full_path += transcribed[0]['date_d'] + '/'
full_path += transcribed[0]['filename']
os.path.isfile(full_path), full_path

# end

In [ ]:
server_object = stt_server(0)
server_object.set_today_ymd()
for source_id in server_object.sources: # ['call', 'master']
    #server_object.source_id = server_object.sources['call']
    server_object.source_id = server_object.get_source_id(source_id)
    complete_files	= server_object.get_sql_complete_files()
    incomplete_count = 0
    complete_count = 0
    print('server_object.source_id', server_object.source_id)
    for filename, rec_date in server_object.get_fs_files_list():
        print(filename, rec_date)
    break

In [ ]:
complete_files

In [ ]:
server_object = stt_server(0)
server_object.source_id = 2
fd_list = server_object.get_fs_files_list()
len(fd_list)

In [ ]:
fd_list[0]

In [ ]:
df = pd.DataFrame(fd_list)
df.sort_values(['rec_date', 'filename'], ascending=False, inplace=True)

In [ ]:
#df[df.rec_date=='Null']
for f,d in df.values:
    print(f,d)
    break

In [ ]:
filename = '1612946589.1564912-in.wav'
uniqueid = re.findall(r'^\d*.\d*', filename)[0]
uniqueid

In [ ]:
cursor = server_object.mysql_conn[server_object.source_id].cursor()
query = "select calldate from cdr where uniqueid = '"+uniqueid+"' limit 1;"
cursor.execute(query)
for row in cursor.fetchall():
    #rec_date = str(row[0])
    print(row[0])

In [ ]:
def get_source_id(source_name):
    for source in server_object.sources.items():
        if source[0] == source_name:
            return source[1]
    return 0
get_source_id('master')

In [ ]:
server_object.rec_date = 'Null'
filename = 'in_4957237230_2021-02-10-07-16-03rxtx.wav'
rec_source_date = re.findall(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}', filename)[0]                
if len(rec_source_date):
    server_object.rec_date = rec_source_date[:10] + ' ' + rec_source_date[11:].replace('-', ':')
if len(re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', server_object.rec_date)) == 0:
    print('Unable to extract date from filename', filename)
server_object.rec_date

In [ ]:
rec_source_date

In [ ]:
server_object.rec_date

In [ ]:
re.findall(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', server_object.rec_date)

In [ ]:
for source_id in server_object.sources:
    print(get_source_id(source_id))

In [4]:
server_object = stt_server(0)
server_object.source_id = 1

In [ ]:
files = server_object.get_fs_files_list()
len(files), files[0]

In [ ]:
files

In [ ]:
complete = server_object.get_sql_complete_files()
len(complete)

### transcribations show columns

In [ ]:
cursor = server_object.conn.cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

### PT1C_cdr_MICO show columns

In [ ]:
cursor = server_object.mysql_conn[1].cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='PT1C_cdr_MICO';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

### tables

In [ ]:
cursor = server_object.mysql_conn[1].cursor()
query = "show tables;"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

### cel columns

In [ ]:
cursor = server_object.mysql_conn[1].cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='cel';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

In [ ]:
cursor = server_object.mysql_conn[1].cursor()
uniqueid = '1613647500.1738521'
query = "select calldate, src, dst from cdr where uniqueid = '" + uniqueid + "' limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

### select from cel

In [ ]:
cursor = server_object.mysql_conn[1].cursor()

sql_query = "select * from cel where linkedid = '1613049028.1603159';"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

### queue show columns

In [77]:
cursor = server_object.conn.cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='queue';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('date',)
('cpu_id',)
('filepath',)
('date_y',)
('date_m',)
('date_d',)
('filename',)
('duration',)
('record_date',)
('source_id',)
('src',)
('dst',)
('linkedid',)


### transcribations show columns

In [5]:
cursor = server_object.conn.cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

('transcribation_date',)
('date_y',)
('date_m',)
('date_d',)
('side',)
('text',)
('start',)
('audio_file_name',)
('conf',)
('end_time',)
('sentiment',)
('sentiment_pos',)
('sentiment_neg',)
('ID',)
('linkedid',)
('dst',)
('record_date',)
('source_id',)
('src',)
('cpu_id',)
('duration',)


### transcribed phrases

In [ ]:
server_object = stt_server(0)
server_object.source_id = 2

cursor = server_object.conn.cursor()

#sql_query = "select audio_file_name, linkedid, record_date, start, side, text from transcribations where source_id = '2' order by record_date, start, side;"
sql_query = "select top 4 conf, src, dst, audio_file_name, side, record_date from transcribations where source_id = '2' group by conf, src, dst, audio_file_name, side, record_date order by record_date desc;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

In [ ]:
cursor = server_object.conn.cursor()

sql_query = "select start, side, src, dst, text from transcribations where linkedid = '1613049028.1603159'"
sql_query += " order by start, side;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

### get record date from cdr

In [ ]:
cursor = server_object.mysql_conn[server_object.source_id].cursor()
query = "SELECT * FROM cdr WHERE uniqueid = '1613049028.1603159';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

In [ ]:
cursor = server_object.mysql_conn[server_object.source_id].cursor()
query = "SELECT column_name FROM information_schema.columns WHERE table_name='cdr';"
cursor.execute(query)
for row in cursor.fetchall():
        print(row)

In [ ]:
server_object.source_id = 2
#server_object.original_file_name = '1612427793.1440877-in.wav'#files[0] #debug
server_object.original_file_name = '1612427793.1440877-in.wav'#files[0] #debug
uniqueid = re.findall(r'^\d*.\d*', server_object.original_file_name)[0]
print('uniqueid', uniqueid)
cursor = server_object.mysql_conn[server_object.source_id].cursor()
#query = "select src, dst from cdr where uniqueid = '"+uniqueid+"' limit 1;"
query = "select calldate, src, dst from cdr where uniqueid = '"+uniqueid+"' limit 1;"
cursor.execute(query)
for row in cursor.fetchall():
    print(row)

### insert into queue debug

In [ ]:
server_object = stt_server(0)
sql_query = "insert into transcribations( audio_file_name, transcribation_date, date_y, date_m, date_d, text, start, end_time, side, conf, linkedid, src, dst, record_date, source_id) values ( '1613048825.1603082-in.wav', '2021-02-11T16:51:28', '2021', '02', '11', 'аллё здравствуйте вячеслав мареку компании единственная служба сервиса звоню по вашему резюме на избита удобно сейчас разговаривать', '6.27', '15.6', '0', '0.8597108235294117', '1613048825.1603082', '', '','None' ,'2');"
#cursor = server_object.conn.cursor()
#cursor.execute(sql_query)
#server_object.conn.commit() # autocommit

### delete from queue

In [46]:
server_object = stt_server(0)
#server_object.source_id = 1
cursor = server_object.conn.cursor()
#sql_query = "delete from queue where source_id = 2;"
sql_query = "delete from queue;"
cursor.execute(sql_query)
server_object.conn.commit() # autocommit

### select from queue

In [47]:
server_object = stt_server(0)
server_object.source_id = 2

cursor = server_object.conn.cursor()
#sql_query = "select count(filename) from queue where source_id = '"+str(server_object.source_id)+"';"
#sql_query = "select distinct cpu_id from queue where source_id = '"+str(server_object.source_id)+"';"
#sql_query = "select filename, record_date, cpu_id, duration, source_id from queue "
#sql_query += "where source_id='" + str(server_object.source_id) + "' "
#sql_query += "where duration=60 "
#sql_query += "order by record_date;"

print('# 1')
sql_query = "select record_date, count(filename) from queue "
sql_query += "where source_id='2' and record_date is Null group by record_date;"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

# 1


In [50]:
server_object = stt_server(0)
server_object.source_id = 2

cursor = server_object.conn.cursor()
#sql_query = "select count(filename) from queue where source_id = '"+str(server_object.source_id)+"';"
#sql_query = "select distinct cpu_id from queue where source_id = '"+str(server_object.source_id)+"';"
#sql_query = "select filename, record_date, cpu_id, duration, source_id from queue "
#sql_query += "where source_id='" + str(server_object.source_id) + "' "
#sql_query += "where duration=60 "
#sql_query += "order by record_date;"

print('# 1')
sql_query = "select top 1 * from queue "
sql_query += "where source_id='1';"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)

print('# 2')
sql_query = "select top 1 * from queue "
sql_query += "where source_id='2' "
sql_query += "order by record_date;"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)
    
print('# count 1')
sql_query = "select count(filename) from queue where source_id = '1';"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)
    
print('# count 2')
sql_query = "select count(filename) from queue where source_id = '2';"
#sql_query += "limit 1;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)
    
print('# cpus')
sql_query = "select distinct cpu_id from queue where cpu_id='30' order by cpu_id;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    #print(row[0], row[1], row[2], row[3])
    print(row)


# 1
# 2
(datetime.datetime(2021, 3, 10, 13, 7, 14), 0, '/mnt/share/audio/MSK_SRVCALL/REC_IN_OUT/', None, None, None, '1615288348.2092415-in.wav', 96.44, datetime.datetime(2021, 3, 9, 14, 12, 28), 2, '5014', '989605074141', '1615288348.2092415')
# count 1
(0,)
# count 2
(1150,)
# cpus


### call queue

In [62]:
server_object = stt_server(0)
server_object.source_id = 1
cursor = server_object.conn.cursor()
#sql_query = "select top 2 * from queue where source_id='1' and duration = 0;"
sql_query = "select top 200 record_date from queue where source_id='2' and duration >5 order by record_date;"
#sql_query = "select count(filename) from queue where source_id='2' and duration >5;"
cursor.execute(sql_query)
for row in cursor.fetchall():    
    print(row)

(datetime.datetime(2021, 2, 16, 10, 42, 35),)
(datetime.datetime(2021, 2, 16, 10, 44, 39),)
(datetime.datetime(2021, 2, 16, 10, 44, 39),)
(datetime.datetime(2021, 2, 16, 10, 46, 6),)
(datetime.datetime(2021, 2, 16, 10, 46, 6),)
(datetime.datetime(2021, 2, 16, 10, 47, 41),)
(datetime.datetime(2021, 2, 16, 10, 47, 41),)
(datetime.datetime(2021, 2, 16, 10, 47, 50),)
(datetime.datetime(2021, 2, 16, 10, 47, 50),)
(datetime.datetime(2021, 2, 16, 10, 48, 29),)
(datetime.datetime(2021, 2, 16, 10, 48, 29),)
(datetime.datetime(2021, 2, 16, 10, 49, 48),)
(datetime.datetime(2021, 2, 16, 10, 49, 48),)
(datetime.datetime(2021, 2, 16, 10, 50, 37),)
(datetime.datetime(2021, 2, 16, 10, 50, 37),)
(datetime.datetime(2021, 2, 16, 10, 51, 2),)
(datetime.datetime(2021, 2, 16, 10, 51, 2),)
(datetime.datetime(2021, 2, 16, 10, 51, 21),)
(datetime.datetime(2021, 2, 16, 10, 51, 21),)
(datetime.datetime(2021, 2, 16, 10, 51, 51),)
(datetime.datetime(2021, 2, 16, 10, 51, 51),)
(datetime.datetime(2021, 2, 16, 10, 52

### delete from transcribations

In [ ]:
cursor = server_object.conn.cursor()
sql_query = "delete from transcribations where source_id = '2';"
#cursor.execute(sql_query)
#server_object.conn.commit() # autocommit

In [ ]:
sql_query

In [ ]:
server_object.get_sql_complete_files()

### select from transcribations

In [ ]:
server_object = stt_server(0)
#server_object.source_id = 1

cursor = server_object.conn.cursor()
sql_query = "select top 1 * from transcribations;"
cursor.execute(sql_query)
for row in cursor.fetchall():
    print(row)

# other

In [ ]:
import datetime

In [ ]:
cur_date = datetime.datetime.now()
DD = datetime.timedelta(days=int(365 / 2))
crop_date = cur_date - DD
cur_date_y = crop_date.strftime("%Y")
cur_date_m = crop_date.strftime("%m")
cur_date_d = crop_date.strftime("%d")
cur_date_y, cur_date_m, cur_date_d

In [ ]:
cur_date = datetime.datetime.now()
cur_date

In [ ]:
DD = datetime.timedelta(days=int(365/2))
crop_date = cur_date - DD
crop_date

In [ ]:
#datetime.datetime.now().year
datetime.datetime.now().strftime("%Y")

In [ ]:
#datetime.datetime.now().month
datetime.datetime.now().strftime("%m")

In [ ]:
cur_date = datetime.datetime.strptime("2021-04-12T07:00:00Z","%Y-%m-%dT%H:%M:%SZ")
DD = datetime.timedelta(days=int(365 / 2))
crop_date = cur_date - DD
#datetime.datetime.strptime(str(datetime.datetime.now()),"%m")
crop_date

In [ ]:
#datetime.datetime.now().day
datetime.datetime.now().strftime("%d")

In [ ]:
import pandas as pd
from init_server import stt_server
from deeppavlov import build_model, configs
import numpy as np

In [ ]:
BATCH_SIZE = 3

In [ ]:
server_object = stt_server(0)

In [ ]:
query = """
    select top """+str(BATCH_SIZE)+""" 
    id,
    text,
    sentiment
    from transcribations 
    where sentiment is NULL and text!=''
    """

In [ ]:
df = pd.read_sql(query, server_object.conn)

model = build_model(configs.classifiers.rusentiment_bert, download=True) #download first time
res = model(df.text)
df['sentiment'] = model(df.text)

In [ ]:
print(df)

In [ ]:
df['sentiment'] = ['neutral','negative','skip']

In [ ]:
df

In [ ]:
for index, row in df.iterrows():
    print(index, row.id,row.sentiment)

In [ ]:
for row in df.values:
    print(row.sentiment)

In [ ]:
for index, row in df.iterrows():
    if row.sentiment == 'negative':
        neg = 1
        pos = 0
    else:
        neg = 0
        pos = 1
    query = "update transcribations set "
    query += "sentiment = '"+row.sentiment+"', "
    query += "sentiment_neg = "+str(neg)+", "
    query += "sentiment_pos = "+str(pos)+" "
    query += "where id = "+str(row.id)
    break

In [ ]:
query

In [ ]:
df['sentiment_neg']=np.zeros(len(df))
df['sentiment_pos']=np.zeros(len(df))

In [ ]:
sentiments = pd.DataFrame(columns=['sentiment','sentiment_neg','sentiment_pos'])

In [ ]:
sentiments = sentiments.append(pd.DataFrame({
    'sentiment':
        [
            'negative',
            'positive',
            'neutral',
            'speech',
            'skip'
        ],
    'sentiment_neg':
        [
            1,0,0,0,0
        ],
    'sentiment_pos':
        [
            0,1,1,1,1
        ]
}))

In [ ]:
sentiments

In [ ]:
df

In [ ]:
df[df.sentiment=='negative'].sentiment_neg=np.ones(len(df[df.sentiment=='negative']))

In [ ]:
df[df.sentiment=='negative']

In [ ]:
df[df.sentiment=='negative'].sentiment_neg=[1]

In [ ]:
df[df.sentiment=='negative'].sentiment_neg

In [ ]:
df.set_value[df.sentiment=='negative']['sentiment_neg']=np.ones(len(df[df.sentiment=='negative']))
#df[df.sentiment!='negative']['sentiment_pos']=1
#df.fillna(0)
#df.set_value('C', 'x', 10)

In [ ]:
df[df.sentiment=='negative']['sentiment_neg']